In [34]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Crear Sesion de Spark
spark = SparkSession.builder.appName("Arquitectura 1").config("spark.sql.legacy.timeParserPolicy", "LEGACY").getOrCreate()

In [35]:
#Se lee el archivo

df = spark.read.csv("TGN_Entrega_GBA_Agosto.csv", header="true",sep=";")

df.show(10)

+----------+----------------+--------------------+--------------+----------+---------------------------+-----------------------------+---------------------+--------+
|     Fecha|      Cargadores|    Punto de Entrega|     Gasoducto|Progresiva|Volumen Nominado (miles m3)|Volumen Autorizado (miles m3)|Volumen CF (miles m3)|Contrato|
+----------+----------------+--------------------+--------------+----------+---------------------------+-----------------------------+---------------------+--------+
|08/01/2023|   30-71166333-5|   Alejandro Roca II|  Centro Oeste|     862,0|                      30,00|                        30,00|                 0,00|   TI457|
|08/01/2023|   30-60080273-5|  Catamarca-La Rioja|         Norte|     852,3|                       5,66|                         5,66|                 5,66|   TF109|
|08/01/2023|   30-61211333-1|  Catamarca-La Rioja|         Norte|     852,3|                      10,00|                        10,00|                10,00|   TF120|
|08/

In [36]:
from pyspark.sql.functions import to_date, month, avg, sum, col, when, regexp_replace, expr

# Convertir la columna 'fecha' al formato de fecha correcto
df1 = df.withColumn('Fecha', when(expr("substring(Fecha, 5, 1) = '/'"), to_date('Fecha', 'dd/MM/yyyy')).otherwise(to_date('Fecha', 'MM/dd/yyyy')))

# Reemplazar comas en 'Volumen Nominado (miles m3)' y convertir a tipo numérico
df1 = df1.withColumn('Volumen Nominado (miles m3)', regexp_replace('Volumen Nominado (miles m3)', ',', '').cast('double'))

# Reemplazar nulos en 'Volumen Nominado (miles m3)' con 0
df1 = df1.withColumn('Volumen Nominado (miles m3)', when(col('Volumen Nominado (miles m3)').isNull(), 0).otherwise(col('Volumen Nominado (miles m3)')))

# Agregar una columna adicional para identificar valores no numéricos
df1 = df1.withColumn('EsNumerico', when(df1['Volumen Nominado (miles m3)'].cast('string').rlike('^[0-9]+\\.?[0-9]*$'), 1).otherwise(0))

# Filtrar registros donde la columna no es numérica
df1 = df1.filter(df1['EsNumerico'] == 1)

result = df1.groupBy(month('Fecha').alias('mes')).agg(
    avg('Volumen Nominado (miles m3)').alias('volumen_nominado_promedio'),
    sum('Volumen Nominado (miles m3)').alias('volumen_nominado_total')
)

print("Volumen Nominado Total por mes")
result.show()

Volumen Nominado Total por mes
+---+-------------------------+----------------------+
|mes|volumen_nominado_promedio|volumen_nominado_total|
+---+-------------------------+----------------------+
|  8|        3471.874452266243|  1.3486149122382995E8|
+---+-------------------------+----------------------+



In [31]:
# Calcular la sumatoria de volúmenes nominados por Punto de Entrega y Gasoducto

result2 = df1.groupBy('Punto de Entrega','Gasoducto').agg(sum('Volumen Nominado (miles m3)').alias('sumatoria_volumen_nominado'))

print("Volumen Nominado Total: Por Gasoducto y Puntos de Entrega")
result2.show(10)

Volumen Nominado Total: Por Gasoducto y Puntos de Entrega
+--------------------+--------------+--------------------------+
|    Punto de Entrega|     Gasoducto|sumatoria_volumen_nominado|
+--------------------+--------------+--------------------------+
|              Casaro|         Norte|        0.8000000000000004|
|       Centro Este 1|         Norte|                   1827.88|
|            Morrison|         Norte|        131.54000000000002|
|             Arteaga|  Centro Oeste|                    336.37|
|Valle de Calamuchita|         Norte|        12592.520000000002|
|        Zona Litoral|Tramos Finales|        116497.95999999977|
|Lib. Gral. San Ma...|         Norte|        1096.2300000000002|
|            Aldea ER|  PetroUruguay|        31552.049999999992|
|Torzalito - C.T. ...|         Norte|        2802.8100000000004|
|            Lamadrid|         Norte|         7846.029999999998|
+--------------------+--------------+--------------------------+
only showing top 10 rows



In [37]:
# Calcular la sumatoria de volúmenes autorizados por Punto de Entrega y Gasoducto

# Reemplazar comas en 'volumen nominado' y convertir a tipo numérico
df3 = df1.withColumn('Volumen Autorizado (miles m3)',regexp_replace('Volumen Autorizado (miles m3)', ',', '').cast('double'))

# Reemplazar nulos en 'volumen nominado' con 0
df3 = df3.withColumn('Volumen Autorizado (miles m3)', when(col('Volumen Autorizado (miles m3)').isNull(), 0).otherwise(col('Volumen Autorizado (miles m3)')))

result3 = df3.groupBy('Punto de Entrega','Gasoducto').agg(sum('Volumen Autorizado (miles m3)').alias('sumatoria_volumen_autorizado'))

print("Volumen Autorizado Total: Por Gasoducto y Puntos de Entrega")
result3.show()

Volumen Autorizado Total: Por Gasoducto y Puntos de Entrega
+--------------------+--------------+----------------------------+
|    Punto de Entrega|     Gasoducto|sumatoria_volumen_autorizado|
+--------------------+--------------+----------------------------+
|              Casaro|         Norte|                        80.0|
|       Centro Este 1|         Norte|                    182788.0|
|            Morrison|         Norte|                     13154.0|
|             Arteaga|  Centro Oeste|                     33051.0|
|Valle de Calamuchita|         Norte|                   1221932.0|
|        Zona Litoral|Tramos Finales|           9201902.784030002|
|Lib. Gral. San Ma...|         Norte|                    109623.0|
|            Aldea ER|  PetroUruguay|                   3148527.0|
|Torzalito - C.T. ...|         Norte|                    151251.0|
|            Lamadrid|         Norte|                    779625.0|
|            Arequito|  Centro Oeste|                     50202.0|
| 

In [38]:
# Calcular la cantidad de entregas por cargador

from pyspark.sql.functions import count

result4 = df3.groupBy('Cargadores').agg(count('Fecha').alias('cantidad_entregas'))

print("Cantidad de Entregas por Cargador")
result4.show()



Cantidad de Entregas por Cargador
+--------------------+-----------------+
|          Cargadores|cantidad_entregas|
+--------------------+-----------------+
|Petroquimica Rio ...|               31|
|Compa�ia Administ...|             5859|
| ENERGY TRADERS S.A.|               78|
|RAFAEL ALBANESI S.A.|             1497|
|     Pluspetrol S.A.|              279|
|       Redengas S.A.|              310|
|LATIN ENERGY GROU...|               37|
|San Atanasio Ener...|              961|
|Camuzzi Gas del S...|               24|
|      TEPSI GAS S.A.|               31|
|  GAS PATAGONIA S.A.|              338|
|      GASMARKET S.A.|               31|
|Distribuidora de ...|             8637|
|       30-60080273-5|               31|
|   Juan Minetti S.A.|               31|
|      COROCOTTA S.A.|              155|
| NATURAL ENERGY S.A.|              137|
|ENERGY CONSULTING...|              183|
| GAS MERIDIONAL S.A.|              341|
|PAN AMERICAN ENER...|              341|
+--------------------+-

In [ ]:
#De PRUEBA: Grafico
"""
import matplotlib.pyplot as plt

# Obtener los resultados
result_df = result4.toPandas()

# Crear un gráfico de barras : DE PRUEBA 
plt.figure(figsize=(12, 6))
bars = plt.bar(result_df['Cargadores'], result_df['cantidad_entregas'])
plt.ylabel('Cargadores')
plt.xlabel('Cantidad de Entregas')
plt.title('Cantidad de Entregas por Cargador')
plt.xticks(rotation=45)
plt.tight_layout()

# Agregar etiquetas en las barras
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, int(yval), va='bottom', ha='center')

# Ajustar el diseño
plt.tight_layout()

# Mostrar el gráfico
plt.show()
"""